<a href="https://colab.research.google.com/github/devloper13/SiameseNetworkProject/blob/master/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from builtins import input

#import system things
from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import os
import keras


# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import tensorflow as tf
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


    100% |████████████████████████████████| 993kB 21.8MB/s 


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
class Siamese:

    # Create model
    def __init__(self):
        self.x1 = tf.placeholder(tf.float32, [None, 64881])
        self.x2 = tf.placeholder(tf.float32, [None, 64881])

        with tf.variable_scope("Siamese") as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        #self.y_ = tf.placeholder(tf.int32, [None])
        #self.loss = self.loss_with_spring()
        self.loss = self.cosineLoss()

    def network(self, x):
        
        x = tf.reshape(x,shape=[-1,1,64881,1])
        activated_conv1 = self.conv_layer('conv_1',x,3)
        maxpool1 = self.maxpool_layer('maxp_1',activated_conv1)
        
        flattened_conv = tf.layers.flatten(maxpool1)   #To be removed
        activated_fc1 = self.fc_layer( "fc1",flattened_conv, 128)
        #activated_fc2 = self.fc_layer("fc2",activated_fc1, 1024)
        #activated_fc3 = self.fc_layer("fc3",activated_fc2, 2)
        
        return activated_fc1
        
    def conv_layer(self,name,inputs,cur_channel):
        #print(inputs.get_shape())
        prev_channel = inputs.get_shape()[-1]
        #print(prev_channel)
        init = tf.variance_scaling_initializer(scale=2.0)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[1,10,prev_channel,cur_channel],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_channel],initializer = init)
        conv = tf.nn.conv2d(inputs,w,strides=[1,1,1,1],padding = "SAME")
        activation = tf.nn.relu(conv+b)
        return activation  
      
    def maxpool_layer(self,name,inputs):
        return tf.nn.max_pool(inputs,ksize=[1,1,100,1],strides=[1,1,100,1],padding="SAME")  
    
    def fc_layer(self,name,inputs,cur_layer):
        print(inputs.get_shape())
        prev_layer = inputs.get_shape()[-1]
        init = tf.truncated_normal_initializer(stddev=0.01)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[prev_layer,cur_layer],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_layer],initializer=init)
        activation = tf.matmul(inputs,w)+b
        return activation


    def loss_with_spring(self):
        margin = 5.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        
        C = tf.constant(margin, name="C")
        
        pos = tf.multiply(labels_t, eucd2, name="yi_x_eucd2")
        
        neg = tf.multiply(labels_f, tf.pow(tf.maximum(tf.subtract(C, eucd), 0), 2), name="Nyi_x_C-eucd_xx_2")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

        
    def cosineLoss(self):
        
        norms1 = tf.norm(self.o1,axis=1)
        norms2 = tf.norm(self.o2,axis=1)
        norm = tf.multiply(norms1,norms2)
        cosines = tf.reduce_sum(tf.multiply(self.o1,self.o2),axis=1)
        
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        
        
        C = tf.constant(0.4, name="C")
        
        pos = tf.multiply(labels_t, cosines, name="yi_x_cosine")
        
        neg = tf.multiply(labels_f, tf.maximum(tf.subtract(cosines,C),0), name="Nyi_x_C-cosine")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss
        

          

In [5]:
link="https://drive.google.com/open?id=1AycFi7m8_NYsK0zTuaAFrci0cLQdE51i"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('three_hash')  
with open('three_hash','rb') as f:
   three_hash_dict = pickle.load(f)
print(len(three_hash_dict))

64881


In [0]:
stop_words = set(stopwords.words('english')) 
def getThreeHash(text):
  vectorizer = CountVectorizer()
  tokenizer=vectorizer.build_tokenizer()
  
  hashes=""
  tokens=tokenizer(text)
  for token in tokens:
    if token not in stop_words:
      tokenModi="#"+token+"#"
      output = list(ngrams(tokenModi, 3))
      for a in output:
        hashes+=(''.join(a))+" "
  
  return(hashes)
# quest = "where do i find hot girls"
# hash_var = getThreeHash(quest)
#print(hash_var)

In [11]:
link="https://drive.google.com/open?id=1tpMcShoAyQ-uxJFz4SMDPzrg8glfbh1Z" #create shareable link of google drive file
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('QA.csv')  
data = pd.read_csv('QA.csv',error_bad_lines=False).values

1tpMcShoAyQ-uxJFz4SMDPzrg8glfbh1Z


In [0]:
#   q_list.append(quesVec)
#   a_list.append(ansVec)
#   q_list,a_list = np.array(q_list),np.array(a_list)

In [0]:
# print(list(q_list[1]).count(1))
# print(q_list.shape)

(2, 64881)


In [0]:
# new_hash_vec = [0]*64881
# vectorizer = CountVectorizer()
# tokenizer=vectorizer.build_tokenizer()
# for token in tokenizer(hash_var):
#   new_hash_vec[three_hash_dict[token]] = 1
# #print(new_hash_vec.count(1))

In [0]:
def trainSiamese(quesVec,ansVec):
  # setup siamese network
  siamese = Siamese();
  train_step = tf.train.GradientDescentOptimizer(0.01).minimize(siamese.loss)

  tf.initialize_all_variables().run()


  for step in range(100):
      batch_x1, batch_y1 = quesVec,np.array([1]*10)
#       print(batch_x1)
#       print(batch_y1)
      batch_x2, batch_y2 = ansVec,np.array([1]*10)
      batch_y = (batch_y1 == batch_y2).astype('float')

      _, loss_v = sess.run([train_step,siamese.loss], feed_dict={
                          siamese.x1: batch_x1,
                          siamese.x2: batch_x2,
                          siamese.y_: batch_y})
      
      if np.isnan(loss_v):
          print('Model diverged with loss = NaN')
          quit()

      if step % 10 == 0:
          print ('step %d: loss %.3f' % (step, loss_v))

      if step % 1000 == 0 and step > 0:
          #saver.save(sess, './model')
          embed = siamese.o1.eval({siamese.x1: mnist.test.images})
          embed.tofile('embed.txt')

  # visualize result
  x_test = mnist.test.images.reshape([-1, 28, 28])
  y_test = mnist.test.labels
  #visualize.visualize(embed, x_test, y_test)

In [33]:
# prepare data and tf.session
# mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
tf.reset_default_graph()
sess = tf.InteractiveSession() 
vectorizer = CountVectorizer()
tokenizer=vectorizer.build_tokenizer()
rowcount = 0
qList,aList = [],[]
for row in data[0:100]:
  rowcount += 1
  quesVec,ansVec = [0]*64881,[0]*64881
  quesHash = getThreeHash(row[1].lower())
  ansHash = getThreeHash(row[3].lower())
#   print(quesHash)
  for token in tokenizer(quesHash.lower()):
     quesVec[three_hash_dict[token]] += 1
  for token in tokenizer(ansHash.lower()):
     ansVec[three_hash_dict[token]] += 1
  qList.append(quesVec)
  aList.append(ansVec)
  if rowcount % 10 == 0:    
    trainSiamese(qList,aList)
    qList,aList = [],[]
sess.close()
    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(?, 1947)
(?, 1947)
22.066473
step 0: loss 22.066
31971.613
12.097725
11.618654
11.158556
10.716678
10.292297
9.884722
9.493287
9.117353
8.756306
step 10: loss 8.756
8.409556
8.076538
7.756706
7.4495416
7.15454
6.8712206
6.5991197
6.3377943
6.0868177
5.8457804
step 20: loss 5.846
5.6142864
5.391961
5.1784396
4.9733734
4.7764273
4.587281
4.405625
4.231162
4.063608
3.9026895
step 30: loss 3.903
3.7481427
3.5997162
3.4571679
3.3202636
3.188781
3.0625052
2.9412303
2.8247573
2.7128973
2.6054664
step 40: loss 2.605
2.50229
2.4031992
2.3080325
2.2166348
2.128856
2.0445533
1.963589
1.8858306
1.8111519
1.7394302
step 50: loss 1.739
1.6705487
1.6043952
1.540861
1.4798429
1.4212412
1.36496
1.3109076
1.2589955
1.2091393
1.1612575
step 60: loss 1.161
1.1152717
1.0711069
1.028691
0.9879549
0.94883186
0.9112581
0.87517226
0.8405155
0.80723107
0.77526474
step 70: loss 0.775
0.74456424
0.7150795
0.68676233
0.6595665
0.63344765
0.60836315
0.584272
0.5611348
0.5389139
0.5175729
step 80: loss 0.518
0.4970

NameError: ignored